# 1. Importing modules

In [1]:
import sys
import os
import numpy as np
import h5py

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Input, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.callbacks import *
from tensorflow.keras.losses import *

# 2. Defining model
![alt text](figures/vgg16-class.png)

In [4]:
def getVGGModel():
    input_tensor = Input(shape=(224,224,3))
    vgg_model = VGG16(weights='imagenet', include_top=False, input_tensor=input_tensor)
    x = vgg_model.get_layer('block5_pool').output

    x = Flatten()(x)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(3, activation='softmax')(x)
    cus_model = keras.Model(inputs=vgg_model.input, outputs=x)

    return cus_model

In [5]:
model = getVGGModel()

# 3. Disabling trained VGG

In [6]:
for layer in model.layers[:19]:
    layer.trainable = False

# 4. Initializing optimizer

In [7]:
opt = keras.optimizers.Adam(lr=0.00001, beta_1=0.9, beta_2=0.999)

# 5. Compiling model

In [8]:
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [9]:
print(model.summary())

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

# 6. Initializing data generator with Data Augmentation

In [10]:
datagen = ImageDataGenerator(
#    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

train_it = datagen.flow_from_directory('/tmp/Dataset_2/Train/', target_size=(224,224), class_mode='categorical', batch_size=16, shuffle=True)
val_it = datagen.flow_from_directory('/tmp/Dataset_2/Validation/', target_size=(224,224), class_mode='categorical', batch_size=1, shuffle=False)

Found 1322 images belonging to 3 classes.
Found 363 images belonging to 3 classes.


# 7.Training with data augmentation
## watch out the val_accuracy compared to train_accuracy, augmentation is an effective method to avoid overfitting

```
Epoch 1/3
83/83 [==============================] - 390s 5s/step - loss: 7.6854 - accuracy: 0.3941 - val_loss: 0.1029 - val_accuracy: 0.4986
Epoch 2/3
83/83 [==============================] - 307s 4s/step - loss: 5.9958 - accuracy: 0.4977 - val_loss: 1.1921e-07 - val_accuracy: 0.6226
Epoch 3/3
83/83 [==============================] - 306s 4s/step - loss: 5.0488 - accuracy: 0.5756 - val_loss: 0.0015 - val_accuracy: 0.5978
```

In [11]:
model.fit_generator(train_it,
                    steps_per_epoch=83,
                    validation_data=val_it,
                    validation_steps=363,
                    epochs=3)

/scratch1/00946/zzhang/python-envs/py3.8-torch1.10/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/3
83/83 [==============================] - 213s 3s/step - loss: 11.9367 - accuracy: 0.3803 - val_loss: 5.2306 - val_accuracy: 0.5482
Epoch 2/3
 1/83 [..............................] - ETA: 2:51 - loss: 8.2234 - accuracy: 0.5000

KeyboardInterrupt: 